### Generate points table

In [1]:
import pandas as pd
import numpy as np

In [2]:
icc_world_cup_data = pd.read_csv('./data/icc_world_cup_data.csv')
icc_world_cup_data

,match_no,team_1,team_2,winner
0,1,ENG,NZ,NZ
1,2,PAK,NED,PAK
2,3,AFG,BAN,BAN
3,4,SA,SL,SA
4,5,AUS,IND,IND
5,6,NZ,NED,NZ
6,7,ENG,BAN,ENG
7,8,SL,PAK,PAK
8,9,AFG,IND,IND
9,10,SA,AUS,SA


In [17]:
def decide_winner(df,team_name):
    if df[team_name]==df['winner']:
        return df[team_name]
    elif df['winner']=='DRAW':
        return 'DRAW'
team1 = icc_world_cup_data[['team_1','winner']].assign(flag = lambda df: df[['team_1','winner']].apply(lambda df: decide_winner(df,'team_1'),axis=1)).rename(columns={'team_1':'team'})
team2 = icc_world_cup_data[['team_2','winner']].assign(flag = lambda df: df[['team_2','winner']].apply(lambda df: decide_winner(df,'team_2'),axis=1)).rename(columns={'team_2':'team'})

In [26]:
concate_data = pd.concat([team1,team2])

In [37]:
concate_data['won'] = (concate_data['team']==concate_data['flag']).replace({True:1,False:0})
concate_data['draw'] = (concate_data['winner']=='DRAW').replace({True:1,False:0})
concate_data.sort_values('team')

,team,winner,flag,won,draw
2,AFG,BAN,None,0,0
8,AFG,IND,None,0,0
9,AUS,SA,None,0,0
4,AUS,IND,None,0,0
6,BAN,ENG,None,0,0
2,BAN,BAN,BAN,1,0
10,BAN,NZ,None,0,0
0,ENG,NZ,None,0,0
6,ENG,ENG,ENG,1,0
8,IND,IND,IND,1,0


In [80]:
result  = concate_data.groupby('team').agg({'won':'sum','draw':['sum','count']}).assign(lost = lambda df: df[('draw','count')]-df[('draw','sum')]-df[('won','sum')])
result

won draw       lost
     sum  sum count     
team                    
AFG    0    0     2    2
AUS    0    0     2    2
BAN    1    0     3    2
ENG    1    0     2    1
IND    3    1     4    0
NED    0    0     2    2
NZ     3    0     3    0
PAK    2    0     3    1
SA     2    1     3    0
SL     0    0     2    2

In [81]:
result.columns

MultiIndex([( 'won',   'sum'),
            ('draw',   'sum'),
            ('draw', 'count'),
            ('lost',      '')],
           )

In [82]:
result.columns = [(i[0]+i[1]) for i in result.columns]
result

,wonsum,drawsum,drawcount,lost
team,,,,
AFG,0,0,2,2
AUS,0,0,2,2
BAN,1,0,3,2
ENG,1,0,2,1
IND,3,1,4,0
NED,0,0,2,2
NZ,3,0,3,0
PAK,2,0,3,1
SA,2,1,3,0


In [90]:
result.rename(columns={'wonsum':'won','drawsum':'draw','drawcount':'total_matches'},inplace=True)
result = result[['total_matches','won','lost','draw']].reset_index()

In [89]:
result

,team,won,lost,draw,total_matches
0,AFG,0,2,0,2
1,AUS,0,2,0,2
2,BAN,1,2,0,3
3,ENG,1,1,0,2
4,IND,3,0,1,4
5,NED,0,2,0,2
6,NZ,3,0,0,3
7,PAK,2,1,0,3
8,SA,2,0,1,3
9,SL,0,2,0,2
